In [1]:
import os
import glob
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr
from datetime import datetime, date
import json
from xgrads import open_CtlDataset
import geopandas as gpd
import requests
import matplotlib.pyplot as plt
from geocube.api.core import make_geocube
%matplotlib inline

# In QGIS

Once you have the huc4 geojson, the streamgage station shapefiles and the filled dem downloaded - you need to do some work in QGIS to dermine the upslope area of each stream gage. 

A detailed description of the workflow can be found here: https://docs.google.com/document/d/1Urdpp41tCZp0ZptHDdRD2HQ3ZznolhWA0Zaa_szcgfc/edit?usp=sharing
Here is the overview:
1. Load DEM and shapefiles
2. Clip dem to HUC4
3. Fill Sinks tool
4. Strahler Order tool
5. Using station locations, use Upslope Area tool to save out .tif files with upslope area for each gage. 
6. Rasterize the huc4 shapefile to use for masking later. 

In [3]:
# Choose a domain
domain = 'BEAU'

# Path to the SnowModel folder
hfdatapath = '/nfs/attic/dfh/2020_NPRB/domain_'+domain+'/model_runs/'
gagepath = '/nfs/attic/dfh/2020_NPRB/data/stream_gage/'+domain+'/'

#path to NPRB domains
domains_resp = requests.get("https://raw.githubusercontent.com/NPRB/02_preprocess_python/main/NPRB_domains.json")
domains = domains_resp.json()

In [17]:
list_of_files = sorted( filter( os.path.isfile,
                        glob.glob(gagepath + '*.tif') ) )

In [18]:
list_of_files

['/nfs/attic/dfh/2020_NPRB/data/stream_gage/BEAU/15798700.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/BEAU/15803000.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/BEAU/15820000.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/BEAU/15860000.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/BEAU/15861000.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/BEAU/15862500.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/BEAU/15875000.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/BEAU/15896000.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/BEAU/15904800.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/BEAU/15905100.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/BEAU/15906000.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/BEAU/15908000.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/BEAU/15955000.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/BEAU/15960000.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/BEAU/15980000.tif']

In [22]:
for i in range(len(list_of_files)):

    file = list_of_files[i]
    stncode=file[-12:-4]
    data = rxr.open_rasterio(file)
    da = data.sel(band=1)
    if i ==0:
        ds = da.to_dataset(name=stncode)
    else:
        ds[stncode]=da
ds

<xarray.Dataset>
Dimensions:      (x: 1298, y: 496)
Coordinates:
    band         int64 1
  * y            (y) float64 2.456e+06 2.454e+06 ... 1.962e+06 1.960e+06
  * x            (x) float64 -5.425e+05 -5.415e+05 ... 7.535e+05 7.545e+05
    spatial_ref  int64 0
Data variables:
    15798700     (y, x) float32 ...
    15803000     (y, x) float32 ...
    15820000     (y, x) float32 ...
    15860000     (y, x) float32 ...
    15861000     (y, x) float32 ...
    15862500     (y, x) float32 ...
    15875000     (y, x) float32 ...
    15896000     (y, x) float32 ...
    15904800     (y, x) float32 ...
    15905100     (y, x) float32 ...
    15906000     (y, x) float32 ...
    15908000     (y, x) float32 ...
    15955000     (y, x) float32 ...
    15960000     (y, x) float32 ...
    15980000     (y, x) float32 ...

In [23]:
ds.to_netcdf(gagepath+'upslope.nc')